$\textbf{Описание задачи}$

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нём теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

$\textbf{Входные данные}$

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

$\textbf{Важно:}$

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества. Если товар не встречался в обучающей выборке, его популярность равна 0. Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров. Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

$\textbf{Задание}$
1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
        сортировка просмотренных id по популярности (частота появления в просмотренных),
        сортировка просмотренных id по покупаемости (частота появления в покупках).
    Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)


In [1]:
from __future__ import division, print_function

import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1. Открытие файлов с данными train и test.

In [2]:
with open('/home/alexander/Рабочий стол/train.txt', 'r') as f:
    sess_train = f.read().splitlines()
with open('/home/alexander/Рабочий стол/test.txt', 'r') as f:
    sess_test = f.read().splitlines()

## 2. Разделение наборов данных по просмотренным и приобретенным.

In [3]:
# Обработка данных из train.txt в удобный массив
sess_train_lp = []
for sess in sess_train:
    look_items, pur_items = sess.split(';')
    look_items = map(int, look_items.split(','))
    if len(pur_items) > 0:
        pur_items = map(int, pur_items.split(','))
    else:
        pur_items = []
    sess_train_lp.append([look_items, pur_items])
    
# Обработка данных из test.txt в удобный массив
sess_test_lp = []
for sess in sess_test:
    look_items, pur_items = sess.split(';')
    look_items = map(int, look_items.split(','))
    if len(pur_items) > 0:
        pur_items = map(int, pur_items.split(','))
    else:
        pur_items = []
    sess_test_lp.append([look_items, pur_items])

In [4]:
sess_train_norm = []
for sess in sess_train:
    look_items, pur_items = sess.split(';')
    look_items = list(map(int, look_items.split(',')))
    if len(pur_items) > 0:
        pur_items = list(map(int, pur_items.split(',')))
    else:
        pur_items = []
    sess_train_norm.append([look_items, pur_items])

In [5]:
look_items

[39047, 102806, 27774]

In [6]:
sess_train_norm

[[[0, 1, 2, 3, 4, 5], []],
 [[9, 10, 11, 9, 11, 12, 9, 11], []],
 [[16, 17, 18, 19, 20, 21], []],
 [[24, 25, 26, 27, 24], []],
 [[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39], []],
 [[42], []],
 [[47, 48, 49], []],
 [[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 68, 67], [67, 60, 63]],
 [[71, 72, 73, 74], []],
 [[76, 77, 78], []],
 [[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86], [86]],
 [[114, 77, 115, 116, 117, 118, 119, 120, 121, 120, 122, 123, 124], []],
 [[129, 130, 131, 132, 133, 134], []],
 [[93, 137, 138, 139, 140, 141, 142, 141, 143, 144], []],
 [[97, 155, 156, 155, 157, 93], []],
 [[163, 87, 129, 164, 129, 130, 165, 166], []],
 [[173, 174, 175, 174, 176], []],
 [[136, 192], []],
 [[195, 196], []],
 [[138, 198, 199, 127], [199]],
 [[206, 207], []],
 [[216, 217], []],
 [[220], []],
 [[223, 224, 225], []],
 [[241, 242, 243, 244, 245, 246, 247, 248, 249, 242], []],
 [[253, 254, 169], []],
 [[262, 263, 262, 264], []],
 [[265,
   266,
   267,
   268,
   269,
   270,
   269,
   

## 3. Создание и сортировка массивов из идентификаторов и количества просмотренных и приобретенных для train. 

In [7]:
# Массив просмотров
sess_train_l = [row[0] for row in sess_train_norm]
sess_train_l_np = np.array( [id_n for sess in sess_train_l for id_n in sess] )

# Массив из идентификаторов и количества просмотров в train
sess_train_l_cnt = np.transpose(np.unique(sess_train_l_np, return_counts=True))

In [8]:
sess_train_l_cnt

array([[     0,      6],
       [     1,      6],
       [     2,      9],
       ...,
       [102804,      1],
       [102805,      1],
       [102806,      1]])

In [9]:
# Массив покупок
sess_train_p = [row[1] for row in sess_train_norm]
sess_train_p_np = np.array( [id_n for sess in sess_train_p for id_n in sess] )

# Массив из идентификаторов и количества покупок в train
sess_train_p_cnt = np.transpose(np.unique(sess_train_p_np, return_counts=True))

In [10]:
sess_train_p_cnt

array([[     5,      1],
       [     6,      2],
       [     7,      2],
       ...,
       [102417,      1],
       [102462,      1],
       [102646,      1]])

In [11]:
# Сортировка массивов просмотров и покупок по количеству
sess_train_l_cnt = sess_train_l_cnt[sess_train_l_cnt[:,1].argsort()][::-1]
sess_train_p_cnt = sess_train_p_cnt[sess_train_p_cnt[:,1].argsort()][::-1]

## 4. Расчет показателей для train с предложениями на основе просмотренных.

In [12]:
def prec_rec_metrics(session, reccomendations, k):
    purchase = 0
    for ind in reccomendations:
        if ind in session:
            purchase += 1 
    precision = purchase / k
    recall = purchase / len(session)
    return(precision, recall)

In [13]:
# Расчет показателей для train с предложениями на основе просмотренных.
prec_at_1_tr_l, rec_at_1_tr_l = [], []
prec_at_5_tr_l, rec_at_5_tr_l = [], []
k1, k5 = 1, 5
for i, sess_p in enumerate(sess_train_p):
    # пропускать сессию без покупок
    if sess_p == []: continue
    
    # id просмотров
    sess_l = sess_train_l[i]

    # сортировка ids индексов просмотров в массиве sess_train_l_cnt 
    # сортировка по количеству просмотров
    l_ind_sess = []
    for j in range(len(list(sess_l))):
        l_ind_sess.append(np.where(sess_train_l_cnt[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(list(sess_l)))
    if num_of_recs_k1 == 0: continue
    recs_k1 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tr_l.append(prec_1)
    rec_at_1_tr_l.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    recs_k5 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tr_l.append(prec_5)
    rec_at_5_tr_l.append(rec_5)

In [14]:
avg_prec_at_1_tr_l = np.mean(prec_at_1_tr_l)
avg_rec_at_1_tr_l = np.mean(rec_at_1_tr_l)
avg_prec_at_5_tr_l = np.mean(prec_at_5_tr_l)
avg_rec_at_5_tr_l = np.mean(rec_at_5_tr_l)

In [15]:
with open('ans1.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tr_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tr_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tr_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tr_l, 2)
    ans1 = ' '.join([r1, p1, r5, p5])
    print('Answer 1:', ans1)
    f.write(ans1)

Answer 1: 0.44 0.51 0.83 0.21


19

## 5. Расчет показателей для train с предложениями на основе приобретенных.

In [16]:
# Расчет показателей для train с предложениями на основе приобретенных.
prec_at_1_tr_p, rec_at_1_tr_p = [], []
prec_at_5_tr_p, rec_at_5_tr_p = [], []
k1, k5 = 1, 5

for i, sess_p in enumerate(sess_train_p):
    # пропустить сессию без покупок
    if sess_p == []: continue
    
    # id просмотров
    sess_l = sess_train_l[i]

    # сортировка ids индексов просмотров в массиве sess_train_p_cnt 
    # сортировка по количеству покупок
    l_ind_sess = []
    for j in range(len(list(sess_l))):
        if sess_l[j] not in sess_train_p_cnt[:,0]: continue
        l_ind_sess.append(np.where(sess_train_p_cnt[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(list(sess_l)), len(l_ind_sess_sorted))
    if num_of_recs_k1 == 0: continue
    recs_k1 = sess_train_p_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tr_p.append(prec_1)
    rec_at_1_tr_p.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(sess_l), len(l_ind_sess_sorted))
    if num_of_recs_k5 == 0: continue
    recs_k5 = sess_train_p_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tr_p.append(prec_5)
    rec_at_5_tr_p.append(rec_5)

In [17]:
avg_prec_at_1_tr_p = np.mean(prec_at_1_tr_p)
avg_rec_at_1_tr_p = np.mean(rec_at_1_tr_p)
avg_prec_at_5_tr_p = np.mean(prec_at_5_tr_p)
avg_rec_at_5_tr_p = np.mean(rec_at_5_tr_p)

In [18]:
with open('ans2.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tr_p, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tr_p, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tr_p, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tr_p, 2)
    ans2 = ' '.join([r1, p1, r5, p5])
    print('Answer 2:', ans2)
    f.write(ans2)

Answer 2: 0.68 0.79 0.93 0.25


19

## 6. Создание и сортировка массивов из идентификаторов и количества просмотренных и приобретенных для test.

In [19]:
# Массив просмотров
sess_test_l = [row[0] for row in sess_test_lp]
sess_test_l_np = np.array( [id_n for sess in sess_test_l for id_n in sess] )

In [20]:
sess_test_l_np

array([     6,      7,      8, ..., 102807, 102808, 102809])

In [21]:
# Массив покупок
sess_test_p = [row[1] for row in sess_test_lp]
sess_test_p_np = np.array( [id_n for sess in sess_test_p for id_n in sess] )

In [22]:
sess_test_p_np

array([   66,    63,   162, ..., 44430, 49815, 21841])

## 7. Расчет показателей для test с предложениями на основе просмотренных.

In [36]:
# Расчет показателей для test с предложениями на основе просмотренных.
prec_at_1_tst_l, rec_at_1_tst_l = [], []
prec_at_5_tst_l, rec_at_5_tst_l = [], []
k1, k5 = 1, 5

for i, sess_p in enumerate(sess_test_p):
    # пропустить сессию без покупок
    if sess_p == []: continue
    
    # looks ids
    sess_l = sess_test_l[i]

    # сортировка ids индексов просмотров в массиве sess_train_l_cnt 
    # сортировка по количеству просмотров
    l_ind_sess = []
    new_ids = []
    for j in range(len(list(sess_l))):
        if sess_l[j] not in sess_train_l_cnt[:,0]:
            new_ids.append(list(sess_l[j]))
            continue
        l_ind_sess.append(np.where(sess_train_l_cnt[:,0] == list(sess_l[j]))[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(list(sess_l)))
    if num_of_recs_k1 == 0: continue
    if l_ind_sess != []:
        recs_k1 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    else:
        recs_k1 = []
    recs_k1 = np.concatenate((np.array(recs_k1, dtype='int64'), np.unique(np.array(new_ids, dtype='int64'))))[:num_of_recs_k1]
    #recs_k1
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tst_l.append(prec_1)
    rec_at_1_tst_l.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(list(sess_l)))
    if num_of_recs_k5 == 0: continue
    if l_ind_sess != []:
        recs_k5 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    else:
        recs_k5 = []
    recs_k5 = np.concatenate((np.array(recs_k5, dtype='int64'), np.unique(np.array(new_ids, dtype='int64'))))[:num_of_recs_k5]
    #recs_k5
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tst_l.append(prec_5)
    rec_at_5_tst_l.append(rec_5)

In [37]:
avg_prec_at_1_tst_l = np.mean(prec_at_1_tst_l)
avg_rec_at_1_tst_l = np.mean(rec_at_1_tst_l)
avg_prec_at_5_tst_l = np.mean(prec_at_5_tst_l)
avg_rec_at_5_tst_l = np.mean(rec_at_5_tst_l)

In [38]:
with open('ans3.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tst_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tst_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tst_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tst_l, 2)
    ans3 = ' '.join([r1, p1, r5, p5])
    print('Answer 3:', ans3)
    f.write(ans3)

Answer 3: nan nan nan nan


15

## 8. Расчет показателей для test с предложениями на основе приобретенных.

In [39]:
def uniquifier(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

In [40]:
# Calculate metrics for test dataset, suggestions based on purchases
prec_at_1_tst_p, rec_at_1_tst_p = [], []
prec_at_5_tst_p, rec_at_5_tst_p = [], []
k1, k5 = 1, 5
for i, sess_p in enumerate(sess_test_p):
    # skip sessions without purchases
    if sess_p == []: continue
    
    # looks ids
    sess_l = sess_test_l[i]

    # sorted looks ids indices in sess_train_p_cnt array
    # sort in accordance with purchases counts
    l_ind_sess = []
    new_ids = []
    for j in range(len(list(sess_l))):
        if sess_l[j] not in sess_train_p_cnt[:,0]:
            new_ids.append(sess_l[j])
            continue
        l_ind_sess.append(np.where(sess_train_p_cnt[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(list(sess_l)))
    if num_of_recs_k1 == 0: continue
    if l_ind_sess != []:
        recs_k1 = sess_train_p_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    else:
        recs_k1 = []
    recs_k1 = np.concatenate((np.array(recs_k1, dtype='int64'), np.array(uniquifier(np.array(new_ids, dtype='int64')))))[:num_of_recs_k1]
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tst_p.append(prec_1)
    rec_at_1_tst_p.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    if l_ind_sess != []:
        recs_k5 = sess_train_p_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    else:
        recs_k5 = []
    recs_k5 = np.concatenate((np.array(recs_k5, dtype='int64'), np.array(uniquifier(np.array(new_ids, dtype='int64')))))[:num_of_recs_k5]
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tst_p.append(prec_5)
    rec_at_5_tst_p.append(rec_5)

In [41]:
avg_prec_at_1_tst_p = np.mean(prec_at_1_tst_p)
avg_rec_at_1_tst_p = np.mean(rec_at_1_tst_p)
avg_prec_at_5_tst_p = np.mean(prec_at_5_tst_p)
avg_rec_at_5_tst_p = np.mean(rec_at_5_tst_p)

In [42]:
with open('ans4.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tst_p, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tst_p, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tst_p, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tst_p, 2)
    ans4 = ' '.join([r1, p1, r5, p5])
    print('Answer 4:', ans4)
    f.write(ans4)

Answer 4: nan nan nan nan


15